## **T-SNE**

T-SNE, which stands for t-Distributed Stochastic Neighbor Embedding, is a statistical method for visualizing high-dimensional data by reducing it to lower dimensions, typically two or three, for easy visualization. It's particularly popular in machine learning and data science for exploring and interpreting complex datasets.

To delve deeper into how t-SNE calculates each feature's position, it's important to understand its iterative process. The algorithm essentially aims to position each data point in the low-dimensional space in a way that preserves the neighborhood relationships from the high-dimensional space. Here's a step-by-step breakdown:

1. **Calculating Pairwise Similarities in High-Dimensional Space**:
   - For each data point in the high-dimensional space, t-SNE computes pairwise similarities.
   - These similarities are the probabilities that one data point would pick another as its neighbor. This probability is proportional to the density of points around a given point, modeled by a Gaussian distribution centered at that point.
   - The similarity of point \( x_j \) to point \( x_i \) is defined as the conditional probability \( p_{j|i} \), which is high if \( x_j \) is close to \( x_i \) and decreases as \( x_j \) moves further away.

2. **Symmetrizing the Similarities**:
   - Since \( p_{j|i} \) is not necessarily equal to \( p_{i|j} \), t-SNE symmetrizes the probabilities using \( p_{ij} = \frac{p_{j|i} + p_{i|j}}{2N} \), where \( N \) is the total number of data points. This step makes sure that the similarity between two points is mutual.

3. **Mapping to a Low-Dimensional Space**:
   - t-SNE then initiates a map in a lower-dimensional space (usually 2D or 3D), where it assigns each high-dimensional object a corresponding low-dimensional representation.
   - Initially, these low-dimensional points are placed randomly.

4. **Calculating Similarities in Low-Dimensional Space**:
   - Similar to the high-dimensional space, t-SNE calculates the similarities in the low-dimensional space. However, it uses a Student's t-distribution (specifically, a t-distribution with one degree of freedom, resembling a Cauchy distribution) instead of a Gaussian distribution. This choice prevents crowding problems and alleviates the curse of dimensionality.
   - In the low-dimensional space, the similarity between two points \( y_i \) and \( y_j \) is represented by \( q_{ij} \).

5. **Minimizing the Kullback-Leibler Divergence**:
   - The goal of t-SNE is to make the similarity distribution in the low-dimensional space (\( q_{ij} \)) as close as possible to that in the high-dimensional space (\( p_{ij} \)).
   - It achieves this by minimizing the Kullback-Leibler divergence between these two distributions, which is a measure of how one probability distribution diverges from a second, expected probability distribution.
   - The KL divergence is minimized using gradient descent. During this process, t-SNE iteratively adjusts the positions of the points in the low-dimensional map.

6. **Iterative Optimization**:
   - The optimization process involves moving points around in the low-dimensional space to reduce the KL divergence. This process is iterative and continues for a specified number of iterations or until a certain convergence criterion is met.
   - Throughout this optimization, t-SNE pays more attention to preserving local structures and relative positions of neighboring points, rather than the global layout.

7. **Result**:
   - The final configuration of points in the low-dimensional space reflects the data's structure in a way that similar objects are modeled by nearby points and dissimilar objects are modeled by distant points.


In [ ]:
from sklearn.manifold import TSNE

# TSNE doesnt work on non-numeric data. Drop the non_numeric column

m = TSNE(learning_rate=50)
tsne_features = m.fit_transform(df_numeric)

df['x'] = tsne_features[:, 0]
df['y'] = tsne_features[:, 1]

import seaborn as sns

sns.scatterplot(x='x', y='y', data=df)
plt.show()

sns.scatterplot(x="x", y="y", hue='BMI_class', data=df)

sns.scatterplot(x="x", y="y", hue='Height_class', data=df)

**Normalize the variance and remove the feature which has low variance**

In [ ]:
from sklearn.feature_selection import VarianceThreshold

sel = VarianceThreshold(threshold=0.005)
sel.fit(ansur_df / ansur_df.mean())

mask = sel.get_support()
reduced_df = ansur_df.loc[:, mask]
print(reduced_df.shape)

In [ ]:
# Fewer than 30% missing values = True value

mask = pokemon_df.isna().sum() / len(pokemon_df) < 0.3
print(mask)

reduced_df = pokemon_df.loc[:, mask]

**LassoCV (It chooses an optimal alpha value)**

In [ ]:
from sklearn.linear_model import LassoCV

# Create and fit the LassoCV model on the training set
lcv = LassoCV()
lcv.fit(X_train, y_train)
print(f'Optimal alpha = {lcv.alpha_:.3f}')

# Calculate R squared on the test set
r_squared = lcv.score(X_test, y_test)
print(f'The model explains {r_squared:.1%} of the test set variance')

# Create a mask for coefficients not equal to zero
lcv_mask = lcv.coef_ != 0
print(f'{sum(lcv_mask)} features out of {len(lcv_mask)} selected')

# **PCA**

Principal Component Analysis (PCA) is a widely used technique in statistics and machine learning for dimensionality reduction, data compression, and exploratory data analysis. The concept of a "Principal Component" is central to understanding PCA. Here's an overview:

### Principal Component

1. **Definition**: A principal component is a direction in the data that maximizes the variance along it. In other words, it's a linear combination of the original variables that captures the maximum amount of information (variance) from the data.

2. **Orthogonality**: Principal components are orthogonal to each other, meaning each component is independent and uncorrelated with the others. This property ensures that each principal component captures a unique aspect of the data's variance.

3. **Ordered by Variance**: The first principal component captures the most variance, the second principal component (which is orthogonal to the first) captures the second most, and so on. The number of principal components is less than or equal to the number of original variables.

4. **Calculation**: Principal components are calculated using an eigenvalue decomposition of the data's covariance matrix or singular value decomposition (SVD) of the data matrix. This process involves finding the eigenvectors (directions of maximum variance) and eigenvalues (amount of variance) of the covariance matrix.

### In PCA

- **Dimensionality Reduction**: PCA transforms the original variables into a new set of variables (the principal components) and ranks them by their importance (variance). By selecting the top few components, you can reduce the dimensionality of the data while preserving most of its variability.
  
- **Data Visualization**: In many cases, especially for high-dimensional data, the first two or three principal components are used to visualize the data in a 2D or 3D space.

- **Noise Reduction**: By ignoring the components with lower variance, PCA can also help in removing noise from the data.

- **Feature Extraction**: PCA is often used for feature extraction, where the principal components become the new features for machine learning models.

### Practical Application

In practice, PCA is used in various fields such as finance (for risk modeling), bioinformatics (for gene expression analysis), image processing, and more. The principal components themselves often do not have a direct, interpretable meaning in the context of the original variables, but their ability to condense information makes them powerful for analysis and predictive modeling.

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
std_df = scaler.fit_transform(df)

from sklearn.decomposition import PCA

pca = PCA()
print(pca.fit_transform(std_df))

**Principal component explained variance ratio**

In [ ]:
from sklearn.decomposition import PCA

pca = PCA()

pca.fit(std_df)
# pc = pca.fit_transform(std_df)

print(pca.explained_variance_ratio_)